# Exploring the representational capacity of FFF

We use CIFAR10 rather than MNIST as MNIST is trite (even a pixel-averaging algo scores pretty high on MNIST)

Conclusion:
- Only training a few epochs, naive PyTorch is getting ~52%, but FFF is getting ~48%.  
- So it's reasonably close!  
- Close enough to warrant further investigation!

In [1]:
import torch as th
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

import sys
if "../FFF" not in sys.path:
    sys.path.insert(0, "../FFF")

%load_ext autoreload
%autoreload 2
from fff import FFF

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1024,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Net_regular(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class Net_FFF(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = FFF(16 * 5 * 5, 120)
        self.fc2 = FFF(120, 84)
        self.fc3 = FFF(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:

def evaluate(net: nn.Module, data_loader: torch.utils.data.DataLoader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in data_loader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
    return 100 * correct / total

def train(net: nn.Module,
          trainloader: torch.utils.data.DataLoader,
          testloader: torch.utils.data.DataLoader,
          epochs: int):

    optimizer = optim.AdamW(net.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch} loss: {running_loss / len(trainloader)}")
        print(f"Train accuracy: {evaluate(net, trainloader)}")
        print(f"Test accuracy: {evaluate(net, testloader)}")

In [5]:
net_regular = Net_regular()
net_FFF = Net_FFF()

print(f'eval net_regular: {evaluate(net_regular, testloader)}% accuracy')
print(f'eval net_FFF: {evaluate(net_FFF, testloader)}% accuracy')

eval net_regular: 10.8100004196167% accuracy
eval net_FFF: 9.770000457763672% accuracy


In [6]:
train(net_regular, trainloader, testloader, 3)

Epoch 0 loss: 1.7733532684233488
Train accuracy: 44.39799880981445
Test accuracy: 44.45000076293945
Epoch 1 loss: 1.4602721588081107
Train accuracy: 50.5359992980957
Test accuracy: 49.79999923706055
Epoch 2 loss: 1.3292798532549377
Train accuracy: 54.108001708984375
Test accuracy: 52.810001373291016


In [7]:
train(net_FFF, trainloader, testloader, 3)

Epoch 0 loss: 2.0134053846149493
Train accuracy: 34.63399887084961
Test accuracy: 34.400001525878906
Epoch 1 loss: 1.653572831617292
Train accuracy: 43.08599853515625
Test accuracy: 43.0099983215332
Epoch 2 loss: 1.5158418740153008
Train accuracy: 48.66400146484375
Test accuracy: 47.29999923706055


In [8]:
%%timeit
evaluate(net_regular, testloader)

12.6 s ± 99.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
evaluate(net_FFF, testloader)

12.7 s ± 64.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
